In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [6]:
data=pd.read_json ('News_Category_Dataset_v2.json', lines=True)

### Typical data analysis of a dataframe now

In [39]:
data.shape


(200853, 6)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200853 entries, 0 to 200852
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   category           200853 non-null  object        
 1   headline           200853 non-null  object        
 2   authors            200853 non-null  object        
 3   link               200853 non-null  object        
 4   short_description  200853 non-null  object        
 5   date               200853 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.2+ MB


In [13]:
#data.describe
data.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [15]:
print(data.groupby('category').size())

category
ARTS               1509
ARTS & CULTURE     1339
BLACK VOICES       4528
BUSINESS           5937
COLLEGE            1144
COMEDY             5175
CRIME              3405
CULTURE & ARTS     1030
DIVORCE            3426
EDUCATION          1004
ENTERTAINMENT     16058
ENVIRONMENT        1323
FIFTY              1401
FOOD & DRINK       6226
GOOD NEWS          1398
GREEN              2622
HEALTHY LIVING     6694
HOME & LIVING      4195
IMPACT             3459
LATINO VOICES      1129
MEDIA              2815
MONEY              1707
PARENTING          8677
PARENTS            3955
POLITICS          32739
QUEER VOICES       6314
RELIGION           2556
SCIENCE            2178
SPORTS             4884
STYLE              2254
STYLE & BEAUTY     9649
TASTE              2096
TECH               2082
THE WORLDPOST      3664
TRAVEL             9887
WEDDINGS           3651
WEIRD NEWS         2670
WELLNESS          17827
WOMEN              3490
WORLD NEWS         2177
WORLDPOST          2579
dtype: 

In [16]:
#we can see that many categories are similar.
#Let´s merge them. We will merge the more specific ones into the more general ones in case of doubt
#the code in the cell below was obtained from https://www.kaggle.com/rmisra/news-category-dataset/discussion/114275

In [18]:
data['category']=data['category'].replace({"HEALTHY LIVING": "WELLNESS",
"QUEER VOICES": "GROUPS VOICES",
"BUSINESS": "BUSINESS & FINANCES",
"PARENTS": "PARENTING",
"BLACK VOICES": "GROUPS VOICES",
"THE WORLDPOST": "WORLD NEWS",
"STYLE": "STYLE & BEAUTY",
"GREEN": "ENVIRONMENT",
"TASTE": "FOOD & DRINK",
"WORLDPOST": "WORLD NEWS",
"SCIENCE": "SCIENCE & TECH",
"TECH": "SCIENCE & TECH",
"MONEY": "BUSINESS & FINANCES",
"ARTS": "ARTS & CULTURE",
"COLLEGE": "EDUCATION",
"LATINO VOICES": "GROUPS VOICES",
"CULTURE & ARTS": "ARTS & CULTURE",
"FIFTY": "MISCELLANEOUS",
"GOOD NEWS": "MISCELLANEOUS"})

In [21]:
print(data.groupby('category').size())

category
ARTS & CULTURE          3878
BUSINESS & FINANCES     7644
COMEDY                  5175
CRIME                   3405
DIVORCE                 3426
EDUCATION               2148
ENTERTAINMENT          16058
ENVIRONMENT             3945
FOOD & DRINK            8322
GROUPS VOICES          11971
HOME & LIVING           4195
IMPACT                  3459
MEDIA                   2815
MISCELLANEOUS           2799
PARENTING              12632
POLITICS               32739
RELIGION                2556
SCIENCE & TECH          4260
SPORTS                  4884
STYLE & BEAUTY         11903
TRAVEL                  9887
WEDDINGS                3651
WEIRD NEWS              2670
WELLNESS               24521
WOMEN                   3490
WORLD NEWS              8420
dtype: int64


In [33]:
(data.groupby('category').size()).std()

7299.533828002569

### As we may observe, the data is still highly imbalanced. Therefore further techniques will have to be applied to
### make up for this

First we split the data such that the proportion of category labels is the same in train and test. Remember, category will be our target

In [38]:
train, test = train_test_split(data, test_size=0.2, stratify=data['category'])

Leave that for later, do not pay too much attention for now

In [37]:

class_weights = dict(zip(np.unique(y_train), class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)))

Then you use the class weights during the training process:

train_history = model.fit( train_dataset, steps_per_epoch=n_steps, class_weight=class_weights )

SyntaxError: invalid syntax (<ipython-input-37-c27e5a755d4f>, line 3)